In [24]:
from typing import List, Tuple, Callable
import math
import heapq


In [25]:
matrix = [
    [True, True , True, False, False, False, False],
    [True, False, True, False, False, False, False],
    [True, False, True, True , True , True , True],
    [True, False, True, False, False, False, True],
    [True, False, True, False, True , True , True],
    [True, False, True, False, True , False, False],
    [True, True , True, True , True , True , True]
]

coordinate = tuple[int, int]

start = (2, 2)
dest = (6,6)



In [26]:
def get_euclidean_distance(pq1: coordinate, pq2: coordinate) -> float:
    p1, q1 = pq1
    p2, q2=  pq2

    return math.sqrt((p1 - p2) ** 2 + (q1 - q2) ** 2)

def is_valid(matrix: List[List[bool]], visited: List[List[bool]], row: int, col: int) -> bool:
    h = len(matrix)
    w = len (matrix)

    # out of bound
    if not ( 0 <= row < h and 0 <= col < w) :
        return False
    
    # invalid node
    if not matrix[row][col] :
        return False
    
    # visied node
    if visited[row][col]:
        return False
    
    return True

def _print_cost(matrix: List[List[int]]) -> None:
    h = len(matrix)
    w = len(matrix[0])

    print("- Heuristic Cost -")
    for i in range(h) :
        for j in range(w) :
            print("." if math.isinf(matrix[i][j]) else matrix[i][j], end = " ")

        print()
    print()

def _print_path(matrix: List[List[bool]], start: coordinate, dest: coordinate, title: str) -> None:
    h = len(matrix)
    w = len(matrix[0])

    print(f"---- {title} -----")
    for i in range(h) :
        for j in range(w) :
            if (i, j) == start:
                print("S", end = " ")
            elif (i, j) == dest:
                print("G", end = " ")
            else:
                print("O" if matrix[i][j] else ".", end = " ")
        print()
    print()


_print_shortest_distance: Callable[[coordinate, coordinate, int], None] = lambda start, dest, total_cost: print(f"{start} -> {dest} shortest distance: {total_cost}")

def _print_shortestdst_path(matrix: List[List[bool]], paths: List[coordinate], start: coordinate, dest: coordinate) -> None:
    h = len(matrix)
    w = len(matrix[0])
    matrix = [["." ] * w for _ in range(h)]

    for i in range(h) :
        for j in range(w):
            if (i, j) == start:
                matrix[i][j] = "S"
            elif (i, j) == dest:
                matrix[i][j] = "G"
                
    prev_y, prev_x = start
    for path in paths:
        cur_y, cur_x = path

        if prev_y < cur_y :
            matrix[cur_y][cur_x] = "↓"
        elif prev_y > cur_y:
            matrix[cur_y][cur_x] = "↑"
        elif prev_x < cur_x:
            matrix[cur_y][cur_x] = "→"
        elif prev_x > cur_x:
            matrix[cur_y][cur_x] = "←"

        prev_y, prev_x = cur_y, cur_x

    print("-- Shortest Path --")
    for i in range(h):
        for j in range(w):
            print(matrix[i][j], end= " ")
        print()
    print()

    


In [27]:
# Direction Vector
d_row = (-1, 0, 1,  0)
d_col = (0,  1, 0, -1)

# up, down, left, right

def a_star(matrix: List[List[int]], start: coordinate, dest: coordinate) -> Tuple[int, List[coordinate]]:
    global d_row
    global d_col

    h = len(matrix[0])
    w = len(matrix[1])

    # heuristic cost table
    heuristic_cost = [[float("inf")] * w for _ in range(h)]

    # get heuristic cost
    for i in range(h) :
        for j in range(w) :
            if matrix[i][j] :
                heuristic_cost[i][j] = round(get_euclidean_distance((i, j), dest))

    row, col = start
    dest_y, dest_x = dest

    visited = [[False] * w for _ in range(h)]

    heap = []
    heapq.heappush(heap, (heuristic_cost[row][col] + 0, row, col))

    total_cost = 0
    came_from = []

    """
        keep loop until heap is empty or get into the destination
        from the heap, take out the cost and marke as visited
        take in the heap after calculation of cost if valid adjacent area exists
    """

    while heap and (row, col) != (dest_y, dest_x) :
        total_cost, row, col = heapq.heappop(heap)

        # Total cost - heuristic cost = able to calculate the real distance from start to current location
        
        depth = total_cost = heuristic_cost[row][col]

        # mark as visited
        visited[row][col] = True

        # if valid adjacent area exists, calculate the cost and put in the heap

        for i in range(4) :
            adj_y = row + d_row[i]
            adj_x = col + d_col[i]
            if is_valid(matrix, visited, adj_y, adj_x):
                total_cost = heuristic_cost[adj_y][adj_x] + depth + 1
                came_from.append(((row, col), (adj_y, adj_x)))

    # came_from
    from_y, from_x = came_from[-1][0]
    paths = []

    for i in range(len(came_from) -1, -1, -1) :
        from_coordinate, to_coordinate = came_from[i]
        to_y, to_x = to_coordinate

        if from_y == to_y and from_x == to_x :
            from_y, from_x = from_coordinate
            paths.insert(0, to_coordinate)

    zj
    return total_cost, paths, visited, heuristic_cost

In [28]:

total_cost, paths, vis, heuristic_cost = a_star(matrix, start, dest)

_print_path(matrix, start, dest, "Path")
_print_cost(heuristic_cost)
_print_path(vis, start, dest, "Visited")
_print_shortest_distance(start, dest, total_cost)
_print_shortestdst_path(matrix, paths, start, dest)

---- Path -----
O O O . . . . 
O . O . . . . 
O . S O O O O 
O . O . . . O 
O . O . O O O 
O . O . O . . 
O O O O O O G 

- Heuristic Cost -
8 8 7 . . . . 
8 . 6 . . . . 
7 . 6 5 4 4 4 
7 . 5 . . . 3 
6 . 4 . 3 2 2 
6 . 4 . 2 . . 
6 5 4 3 2 1 0 

---- Visited -----
. . . . . . . 
. . . . . . . 
. . S . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . G 

(2, 2) -> (6, 6) shortest distance: 12
-- Shortest Path --
. . . . . . . 
. . . . . . . 
. . S . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . G 

